In [1]:
import tensorflow as tf
from keras import layers, models, callbacks
import numpy as np
import matplotlib.pyplot as plt

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1,28,28,1).astype('float32') / 255.0
x_test  = x_test.reshape(-1,28,28,1).astype('float32') / 255.0

y_train_oh = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test_oh  = tf.keras.utils.to_categorical(y_test,  num_classes=10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
def build_cnn():
    model = tf.keras.Sequential([
        layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),

        layers.Dense(10, activation='softmax')
    ])
    return model

In [4]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)
lr_schedule1 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
lr_schedule2 = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.01 * 0.95**epoch, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.15)


model = build_cnn()
model.compile(optimizer=optimizer, loss=loss_fn , metrics=['accuracy'])

model.fit(x_train, y_train_oh, epochs=50, batch_size=256, validation_split=0.1, callbacks=[lr_schedule2, early_stop])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 109s 507ms/step - accuracy: 0.8449 - loss: 1.1922 - val_accuracy: 0.1372 - val_loss: 2.2608 - learning_rate: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.0095.
Epoch 2/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 140s 499ms/step - accuracy: 0.9800 - loss: 0.8189 - val_accuracy: 0.9403 - val_loss: 1.0470 - learning_rate: 0.0095

Epoch 3: LearningRateScheduler setting learning rate to 0.009025.
Epoch 3/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 108s 511ms/step - accuracy: 0.9855 - loss: 0.7952 - val_accuracy: 0.9892 - val_loss: 0.7596 - learning_rate: 0.0090

Epoch 4: LearningRateScheduler setting learning rate to 0.00857375.
Epoch 4/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 140s 501ms/step - accuracy: 0.9884 - loss: 0.7847 - val_accuracy: 0.9915 - val_loss: 0.7423 - learning_rate: 0.0086

Epoch 5: LearningRateScheduler setting learning rate to 0.0081450625.
Epoch 5/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 1

In [5]:
test_loss, test_acc = model.evaluate(x_test, y_test_oh)
print(f"Test accuracy: {test_acc*100:.2f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.9915 - loss: 0.7293
Test accuracy: 99.33
